In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip uninstall dill
!pip install dill==0.3.4

Found existing installation: dill 0.3.6
Uninstalling dill-0.3.6:
  Would remove:
    /usr/local/bin/get_objgraph
    /usr/local/bin/undill
    /usr/local/lib/python3.8/dist-packages/dill-0.3.6.dist-info/*
    /usr/local/lib/python3.8/dist-packages/dill/*
Proceed (y/n)? y
  Successfully uninstalled dill-0.3.6
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached dill-0.3.4-py2.py3-none-any.whl (86 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.14 requires dill>=0.3.6, but you have dill 0.3.4 which is incompatible.
evaluate 0.4.0 requires datasets>=2.0.0, but you have datasets 1.0.2 which is incompatible.


## Train

In [ ]:
%%capture
!pip install datasets evaluate transformers[sentencepiece]
!pip install rouge_score

In [ ]:
# Install the vncorenlp python wrapper
#!pip install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
# !mkdir -p vncorenlp/models/wordsegmenter
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
# !mv VnCoreNLP-1.1.1.jar vncorenlp/ 
# !mv vi-vocab vncorenlp/models/wordsegmenter/
# !mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/
!pip install datasets==1.0.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 4.4 MB/s 
  Attempting uninstall: datasets
    Found existing installation: datasets 2.7.1
    Uninstalling datasets-2.7.1:
      Successfully uninstalled datasets-2.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
evaluate 0.4.0 requires datasets>=2.0.0, but you have datasets 1.0.2 which is incompatible.


In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, TrainingArguments, Seq2SeqTrainingArguments, AutoModel
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/prophetnet-large-uncased")  
model = AutoModelForSeq2SeqLM.from_pretrained("microsoft/prophetnet-large-uncased")

model.to('cuda')

/usr/local/lib/python3.8/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


ProphetNetForConditionalGeneration(
  (prophetnet): ProphetNetModel(
    (word_embeddings): Embedding(30522, 1024, padding_idx=0)
    (encoder): ProphetNetEncoder(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): ProphetNetPositionalEmbeddings(512, 1024, padding_idx=0)
      (embeddings_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0): ProphetNetEncoderLayer(
          (self_attn): ProphetNetAttention(
            (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (feed_forward): ProphetNetFeedForward(
            (activatio

In [ ]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["inputs"], max_length=1024, truncation=True, padding=True
    )
    
    labels = tokenizer(
        examples["labels"], max_length=64, truncation=True, padding=True
    )
    model_inputs['labels'] = labels['input_ids']
    model_inputs['input_ids'] = model_inputs['input_ids']
    return model_inputs

In [ ]:
#data = pd.read_excel("/content/drive/MyDrive/D:  /Đại học/Năm 3/Kỳ 1/Xử lí ngôn ngữ tự nhiên/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Data/data_Tuoi_Tre.xlsx")
#data = pd.read_excel("/content/drive/MyDrive/data_Tuoi_Tre.xlsx")
#data = pd.read_excel("/content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Data/data_Tuoi_Tre.xlsx")
#data = pd.read_excel("/content/drive/MyDrive/NLP/Data/data_Tuoi_Tre.xlsx")
#data = data.dropna()

train = pd.read_excel("/content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Data/Data4model/train.xlsx")
val = pd.read_excel("/content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Data/Data4model/val.xlsx")
test = pd.read_excel("/content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Data/Data4model/test.xlsx")

In [ ]:
#data = data.iloc[:100, :]

In [ ]:
input_lines = train["Content"].values
label_lines = train["Summary"].values

for i in range(len(input_lines)):
  input_lines[i] = str(input_lines[i]) + '</s>'

dict_obj = {'inputs': input_lines, 'labels': label_lines}
dataset = Dataset.from_dict(dict_obj)
train_tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=8)

AttributeError: ignored

In [ ]:
input_lines = val["Content"].values
label_lines = val["Summary"].values

for i in range(len(input_lines)):
  input_lines[i] = str(input_lines[i]) + '</s>'

dict_obj = {'inputs': input_lines, 'labels': label_lines}
dataset = Dataset.from_dict(dict_obj)
val_tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=8)

In [ ]:
input_lines = test["Content"].values
label_lines = test["Summary"].values

for i in range(len(input_lines)):
  input_lines[i] = str(input_lines[i]) + '</s>'

dict_obj = {'inputs': input_lines, 'labels': label_lines}
dataset = Dataset.from_dict(dict_obj)
test_tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=8)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

training_args = Seq2SeqTrainingArguments("/content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/Prophetnet-Large/",
                                      do_train=True,
                                      do_eval=True,
                                      num_train_epochs=3,
                                      learning_rate=1e-4,
                                      warmup_ratio=0.05,
                                      weight_decay=0.01,
                                      per_device_train_batch_size=4,
                                      per_device_eval_batch_size=4,
                                      gradient_accumulation_steps= 1,
                                      logging_dir='./log',
                                      group_by_length=True,
                                      save_strategy="steps",
                                      save_steps= 2242,  
                                      save_total_limit=1,
                                      overwrite_output_dir=True,
                                      fp16=True,
                                      )


In [ ]:
import torch
torch.cuda.empty_cache()
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_datasets,
    data_collator=data_collator,
    eval_dataset= val_tokenized_datasets,
)

trainer.train()

Using cuda_amp half precision backend
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 29893
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 11211
  Number of trainable parameters = 391321600
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
500,3.870600
1000,3.172000
1500,2.967700
2000,2.845500
2500,2.763100
3000,2.663900
3500,2.642500
4000,2.548700
4500,2.457000
5000,2.418700


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible 

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-991fe50fe392>", line 11, in <module>
    trainer.train()
  File "/usr/local/lib/python3.8/dist-packages/transformers/trainer.py", line 1527, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.8/dist-packages/transformers/trainer.py", line 1775, in _inner_training_loop
    tr_loss_step = self.training_step(model, inputs)
  File "/usr/local/lib/python3.8/dist-packages/transformers/trainer.py", line 2523, in training_step
    loss = self.compute_loss(model, inputs)
  File "/usr/local/lib/python3.8/dist-packages/transformers/trainer.py", line 2555, in compute_loss
    outputs = model(**inputs)
  File "/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py", line 1190, in _call_impl
    return forward_call(*input, **kwargs)
  File "/usr/local/l

KeyboardInterrupt: ignored

In [ ]:
import torch
#torch.cuda.empty_cache()
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_datasets,
    data_collator=data_collator,
)
trainer.train(resume_from_checkpoint="/content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/Prophetnet-Large/checkpoint-8968")

Using cuda_amp half precision backend
Loading model from /content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/Prophetnet-Large/checkpoint-8968.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 29893
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 11211
  Number of trainable parameters = 391321600
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 2
  Continuing training from global step 8968
  Will skip the first 2 epochs then the first 1494 batches in the first epoch

  0%|          | 0/1494 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
9000,2.035900
9500,2.144600
10000,2.278800
10500,2.315100
11000,2.306400


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...

TrainOutput(global_step=11211, training_loss=0.4528274563353915, metrics={'train_runtime': 5218.5626, 'train_samples_per_second': 17.185, 'train_steps_per_second': 2.148, 'total_flos': 1.9781367423093965e+17, 'train_loss': 0.4528274563353915, 'epoch': 3.0})

## Inference

In [ ]:
from datasets import load_metric
metric = load_metric("rouge")

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/Prophetnet-Large/checkpoint-11210")
model.to('cuda')

OutOfMemoryError: ignored

In [ ]:
import torch 
import numpy as np
import datasets
metrics = datasets.load_metric('rouge')

max_target_length = 256
dataloader = torch.utils.data.DataLoader(test_tokenized_datasets, collate_fn=data_collator, batch_size=32)

predictions = []
references = []
for i, batch in enumerate(tqdm(dataloader)):
  outputs = model.generate(
      input_ids=batch['input_ids'].to('cuda'),
      max_length=max_target_length,
      attention_mask=batch['attention_mask'].to('cuda'),
  )
  with tokenizer.as_target_tokenizer():
    outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs]

    labels = np.where(batch['labels'] != -100,  batch['labels'], tokenizer.pad_token_id)
    actuals = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in labels]
  predictions.extend(outputs)
  references.extend(actuals)
  metrics.add_batch(predictions=outputs, references=actuals)


metrics.compute()


In [ ]:
# load rouge for validation
import datasets
rouge = datasets.load_metric("rouge")
print("ROUGE 1 SCORE: ",rouge.compute(predictions=predictions, references=references, rouge_types=["rouge1"])["rouge1"].mid)
print("ROUGE 2 SCORE: ",rouge.compute(predictions=predictions, references=references, rouge_types=["rouge2"])["rouge2"].mid)
print("ROUGE L SCORE: ",rouge.compute(predictions=predictions, references=references, rouge_types=["rougeL"])["rougeL"].mid)

In [ ]:
for i in range(1,11):
  print('Prediction: ',predictions[i])
  print('Truth: ',label_lines[i])
  print('Content: ',input_lines[i])

In [ ]:
input_lines[i]


'Điều quan trọng nhất bạn có thể làm là giữ bình tĩnh và tự chủ trong tình huống khẩn cấp. Đôi khi bạn không thể làm gì, và như vậy cũng không sao. Đừng bận lòng khi phải thừa nhận rằng bạn không giúp gì được.  Nếu những người khác có mặt ở hiện trường rối trí hoặc sợ hãi, bạn hãy trấn an họ và huy động mọi người cùng hỗ trợ. Tốt hơn là bạn nên ở bên cạnh người bị nạn với cử chỉ ân cần thay vì có những hành động có thể gây tổn hại thêm. Nếu không chắc phải làm gì, bạn chỉ cần ở lại với nạn nhân. Nếu có thể, bạn hãy bắt mạch cho nạn nhân, ghi nhớ các sự kiện xảy ra và hỏi về bệnh sử của họ. Đây là những thông tin mà có thể bạn cần biết để báo lại với đội cứu hộ. Người ta thường suy nghĩ và hành động hoảng loạn khi đối mặt với tình huống khẩn cấp. Thay vì phản ứng ngay, bạn hãy dành chút thời gian lấy lại bình tĩnh. Hít thở thật sâu trước khi bắt tay vào hành động.  Sự việc thường thay đổi bất ngờ trong tình huống khẩn cấp. Đừng hoảng hốt nếu sự việc không diễn biến như bạn dự tính. Nghỉ

In [ ]:
sentence = """
Chương trình hành động của Chính phủ đã đề ra 33 nhiệm vụ tập trung vào 5 nhóm nhiệm vụ giải pháp, trước hết là nhóm nhiệm vụ nâng cao, thống nhất nhận thức về đặc thù của đô thị, vai trò, vị thế của đô thị trong sự phát triển chung. Trong đó, xác định phát triển đô thị gồm 3 trụ cột chính là quy hoạch, xây dựng và quản lý đô thị.

"Quy hoạch phải đi trước một bước với tư duy đột phá, tầm nhìn chiến lược. Quy hoạch lúng túng, chậm chạp, không được đầu tư ngang tầm thì lãng phí nguồn lực, phát triển không bền vững, phát triển không đột phá. Công tác quy hoạch làm tốt thì mới tiết kiệm được nguồn lực, xác định và phát huy được tiềm năng khác biệt, cơ hội nổi trội, lợi thế cạnh tranh, chỉ ra và hóa giải những khó khăn, thách thức của địa phương, của vùng”, Thủ tướng phân tích.

Theo Thủ tướng, quy hoạch tổng thể nhưng thực hiện có thể phân kỳ phù hợp với nguồn lực; có thể kiên trì thực hiện trong 5 năm, 10 năm, 20 năm, thậm chí hàng trăm năm… và nếu tôn trọng, làm theo quy hoạch hoàn chỉnh thì sẽ có một đô thị trật tự và phát triển.

Ngoài ra cần đầu tư phát triển hệ thống hạ tầng đô thị đồng bộ, hiện đại trên cơ sở huy động mạnh mẽ các nguồn lực, có cơ chế, chính sách để huy động nguồn lực, kết hợp hài hòa, hợp lý, hiệu quả giữa nguồn lực bên trong và bên ngoài, giữa nguồn lực Nhà nước với các nguồn lực xã hội; đồng thời tiết kiệm, tăng thu, giảm chi, phân bổ và sử dụng nguồn lực hiệu quả, đầu tư có trọng tâm, trọng điểm.

“Hiện vẫn còn nhiều vướng mắc về thể chế, cơ chế, chính sách cần xử lý nhưng với cơ chế đang có thì các địa phương vẫn có thể làm được nếu lãnh đạo các bộ, ngành, địa phương tập trung suy nghĩ, vận dụng tối đa các quy định hiện có trên tinh thần dám nghĩ, dám làm", Thủ tướng lưu ý.
Bùi Thạc Chuyên cho biết gặp áp lực khi chuyển thể văn học từ các tác phẩm của Nguyễn Ngọc Tư. Theo anh, truyện của nữ nhà văn thường có lối viết ngắn nhưng giàu tính gợi mở, các nhân vật có số phận, chứa đựng nội tâm và tiếng nói mạnh mẽ. Do đó, ngoài yếu tố sáng tạo, anh cố gắng để giữ lại những gì mình tâm đắc trong truyện gốc. 

Tro tàn rực rỡ có sự tham gia của Phương Anh Đào, Lê Công Hoàng, Bảo Ngọc Doling, Ngô Quang Tuấn, Hạnh Thúy… Phim là câu chuyện của ba phụ nữ với 3 số phận khác nhau, cách họ sống với niềm vui, nỗi ẩn ức và ngọn lửa tình yêu với chính những người đàn ông của đời mình. Bối cảnh của chuyện xảy ra ở xóm Thơm Rơm, Cà Mau với nhiều chất liệu điện ảnh và văn hóa.

Chia sẻ với VietNamNet, Phương Anh Đào cho biết đây là vai diễn thử thách lớn nhất trong sự nghiệp diễn xuất. Trong phim, diễn viên đảm nhận vai Nhàn - một trong 3 nữ chính. Nhàn có số phận tréo ngoe, luôn khao khát yêu và được yêu đến cháy bỏng. Vốn được biết đến với nét đẹp hiện đại, song nữ diễn viên đã “lột xác” ấn tượng với vai diễn người phụ nữ khắc khổ, tảo tần trong phim.

“Tôi thay đổi hình ảnh rất nhiều cho vai diễn với mong muốn mang đến một Phương Anh Đào khác trên màn ảnh. Thay vì những cảnh bầm dập thể xác, nhân vật của tôi chứa nhiều thương tổn tinh thần. Mọi cảm xúc đều được giữ ở bên trong, và nó khiến nhiều người suy ngẫm. Đó cũng là cách kể chuyện của nhân vật mà anh Bùi Thạc Chuyên muốn hướng đến trong phim”, cô chia sẻ. 

Phim ghi nhận điểm sáng với diễn xuất của các diễn viên. Họ lột tả được nhân vật của mình khi bị đẩy vào hoàn cảnh cùng cực và cách thích nghi một cách bị động trước cuộc đời gần như không lối thoát. 

Hai diễn viên nam Lê Công Hoàng và Quang Tuấn thể hiện tròn vai giữa bối cảnh câu chuyện hầu hết đều xoay quanh những người đàn bà. Diễn viên Quang Tuấn tiết chế được biểu cảm cường điệu so với các vai diễn trước đây. Anh khắc họa một gã đàn ông đau đớn mất con gái và nỗi ám ảnh thường trực vì ánh sáng rực rỡ của những ngọn lửa được đốt lên từ chính ngôi nhà mình. Nam diễn viên diễn chừng mực, không lên gân hay gào thét để người xem tự thấu cảm về nỗi đau cùng cực.

"Do quá nhập vai, tôi đã tự lấy bật lửa đốt tay mình và đến giờ vẫn còn để lại vết bỏng", anh kể. Trước ngày phim bấm máy, nam diễn viên dành một tháng để đến set quay, học nghề ở các lò than gần một tháng để đảm bảo hóa thân tốt. 

"""

text =  sentence + " </s>"
encoding = tokenizer(text, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")
outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=256,
    early_stopping=False
)
for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(line)

Thủ tướng Phạm Minh Chính nhấn mạnh việc coi trọng và đầu tư thích đáng cho công tác quy hoạch; đa dạng hóa, kết hợp, phát huy mạnh mẽ các nguồn lực, có cơ chế, chính sách để huy động mạnh mẽ các nguồn lực, kết hợp hài hòa, hợp lý, hiệu quả giữa nguồn lực bên trong và bên ngoài, giữa nguồn lực với các nguồn lực xã hội.


''